In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
i = 0

In [3]:
from image_generation import NutritionLabelGenerator
from augmentation import *

working_dir = "ntbk_test"

generator = NutritionLabelGenerator(output_dir=working_dir)

label_true = generator.generate_nutrition_data()
img_name = f"label_{i:04d}.png"
img_path = generator.generate_image(label_true, img_name)

augment = AugmentationParams.random()
img = cv2.imread(img_path)
img = apply_augmentations(img, augment)
cv2.imwrite(img_path, img)

i = i + 1


In [ ]:
from augmentation import *
from models import *
from deskewing_advanced import DeskewingPipeline
import os
import easyocr
from label_extraction_core.label_extractors import levenshtein_matching_extractor as lme
from label_extraction_core.label_extractors import regex_matching_extractor as rme

deskewer = DeskewingPipeline(debug=True, debug_dir=working_dir)
img_corrected, rot_est = deskewer.deskew(img, method="hough_lines")

print("rotation:", rot_est)

working_dir = "ntbk_test"
cv2.imwrite(os.path.join(working_dir, f"label_{i:04d}_deskewed.png"), img_corrected)



# Run OCR
reader = easyocr.Reader(['en'], gpu=False)
ocr_results = reader.readtext(img_corrected)

bbox_img = draw_bounding_boxes(img_corrected, ocr_results)

cv2.imwrite(os.path.join(working_dir, f"label_{i:04d}_boxes.png"), bbox_img)

import regex_matching as rem

rem.pretty_print_easyocr(ocr_results)

text_in_lines = rem.easyocr_to_lines(ocr_results)
label_pred = rem.extract_nutrients(text_in_lines, rem.nutrient_aliases)

print(label_true)
print(label_pred)


Using CPU. Note: This module is much faster with a GPU.


[[[-487.           3.0019662]]

 [[-484.           3.0019662]]

 [[-492.           3.0194197]]]
rotation: RotationResult(angle=0.0, confidence=0.0, method='hough_lines')
Index  Bounding Box (x,y)                            Text                 Confidence
-------------------------------------------------------------------------------------
0      (296,260), (324,260), (324,286), (296,286)    60}                  0.118877
1      (218,290), (248,290), (248,320), (218,320)    Fet                  0.031452
2      (252,360), (284,360), (284,392), (252,392)    92                   0.296708
3      (249,451), (267,451), (267,473), (249,473)    9                    0.531343
4      (206,146), (277,194), (264,214), (193,165)    utriant              0.618629
5      (173,162), (234,204), (219,226), (158,184)    Calores              0.391357
6      (161,194), (220,237), (205,258), (146,215)    botal tat            0.090472
7      (158,232), (231,287), (215,308), (142,253)    'Saturated           0.17

In [6]:
from augmentation import *
from deskewing_advanced import DeskewingPipeline
import os
from label_extraction_core.label_extractors import levenshtein_matching_extractor as lme
from label_extraction_core.label_extractors import regex_matching_extractor as rme

working_dir = "ntbk_test"

deskewer = DeskewingPipeline(debug=True, debug_dir=working_dir)
img_corrected, rot_est = deskewer.deskew(img, method="hough_lines")

print("rotation:", rot_est)


cv2.imwrite(os.path.join(working_dir, f"label_{i:04d}_deskewed.png"), img_corrected)

regex_matching_extr = rme.RegexMatchingExtractor()
levenshtein_extr = lme.LevenshteinMatchingExtractor(debug=True)

label_pred, ocr_results = levenshtein_extr.predict_from_image(img_corrected)

bbox_img = draw_bounding_boxes(img_corrected, ocr_results, font_scale=1)
cv2.imwrite(os.path.join(working_dir, f"label_{i:04d}_boxes.png"), bbox_img)

print(label_true)
print(label_pred)


Using CPU. Note: This module is much faster with a GPU.


[[[ 724.           1.5009831]]

 [[ -14.           3.0717795]]

 [[ 495.           1.5009831]]

 [[ 387.           1.5009831]]

 [[ 552.           1.5009831]]

 [[ 559.           1.5009831]]

 [[ 616.           1.5009831]]

 [[ 444.           1.5009831]]

 [[ 731.           1.5009831]]

 [[ 380.           1.5009831]]

 [[ 673.           1.5009831]]

 [[ 266.           1.5009831]]

 [[ 609.           1.5009831]]

 [[ 438.           1.5009831]]

 [[ 330.           1.5009831]]

 [[ 502.           1.5009831]]

 [[-528.           3.0717795]]

 [[ 666.           1.5009831]]

 [[-267.           3.0717795]]

 [[-275.           3.0717795]]

 [[ -21.           3.0717795]]

 [[-521.           3.0717795]]

 [[ 732.           1.4835298]]

 [[ 378.           1.5184364]]

 [[ 722.           1.5184364]]

 [[ 607.           1.5184364]]

 [[ 665.           1.5184364]]

 [[ 549.           1.5184364]]

 [[ 493.           1.5184364]]

 [[ 675.           1.4835298]]

 [[ 436.           1.5184364]]]
Lines_0 

Using CPU. Note: This module is much faster with a GPU.
d:\Aleksa\master\nutrition-label-recognition\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Index  Bounding Box (x,y)                            Text                 Confidence
-------------------------------------------------------------------------------------
0      (114,260), (244,260), (244,292), (114,292)    Nutrient             0.976438
1      (388,260), (476,260), (476,288), (388,288)    Valuo                0.914153
2      (72,318), (184,318), (184,346), (72,346)      Calories             0.754798
3      (328,316), (448,316), (448,348), (328,348)    206 kcal             0.830032
4      (76,374), (188,374), (188,406), (76,406)      otal fat             0.966615
5      (98,432), (282,432), (282,464), (98,464)      Soturated fat        0.442161
6      (74,492), (262,492), (262,524), (74,524)      Carbonydrate         0.861529
7      (325,485), (422,485), (422,527), (325,527)    16.2 9               0.816678
8      (95,545), (273,545), (273,584), (95,584)      Dietary fiber        0.757831
9      (96,604), (199,604), (199,642), (96,642)      Sugars               0.999332